In [1]:
using Revise, Pkg
Pkg.activate(".")

In [2]:
using Napari, ThreeDeconv, FFTW, Images, Printf, Statistics

┌ Info: napari version
│   version = 0.4.4
└ @ Napari /home/fxw/.julia/packages/Napari/1a3LQ/src/Napari.jl:48
┌ Info: /home/fxw/julia/PythonEnvs/python38/lib/python3.8/site-packages/napari
└ @ Napari /home/fxw/.julia/packages/Napari/1a3LQ/src/Napari.jl:49
┌ Info: Precompiling ThreeDeconv [8923caa7-4272-43b5-844b-8c6a04cef442]
└ @ Base loading.jl:1317


In [3]:
img = Float32.(load("data.tif"));
psf = Float32.(load("psf.tif"));

┌ Warning: some versions of ImageMagick give spurious low-order bits for 32-bit TIFFs
└ @ ImageMagick /home/fxw/.julia/packages/ImageMagick/b8swT/src/ImageMagick.jl:100


In [4]:
@view_image psf

PyObject <napari.viewer.Viewer object at 0x7f9900142910>

In [5]:
@view_image img

PyObject <napari.viewer.Viewer object at 0x7f98d48323a0>

In [4]:
img ./= maximum(img);

In [11]:
γ, σ = ThreeDeconv.noise_estimation(img, maxnum_pairs = 200)
@printf "Gain: %.1f, Read noise std.: %.1f \n" γ σ


options =
    ThreeDeconv.DeconvolutionOptions(max_iters = 100, show_trace = true, check_every = 50)
reg = 1e-7

Computing local noise variance.
Initializing parameters by least-squares.
Initialization done.
Starting likelihood maximization.
Finished the maximization.
Estimated noise parameters are a = 0.001, b = 0.000.
Estimated parameters for Poisson-Gaussian noise model
γ: 0.0011905210267543537, σ: 0.00888126209563958
Gain: 0.0, Read noise std.: 0.0 


1.0e-7

In [13]:
@time result = ThreeDeconv.deconvolve(img, psf, γ, σ, reg, options = options)

typeof(method) = ThreeDeconv.ADMM
No clipping performed.
Iter     rel_diff      time  
------   ---------   --------
     0         NaN       0.00
    50      0.0729      11.56
   100      0.0518      15.86
 17.906101 seconds (25.59 M allocations: 2.182 GiB, 3.21% gc time, 3.57% compilation time)


ThreeDeconv.OptimizationResult(Float32[203.47272 200.15808 … 158.22076 184.75516; 218.69745 222.63017 … 168.18898 194.63396; … ; 171.86462 163.10675 … 152.17499 167.90913; 184.90424 176.85468 … 152.06786 174.1256]

Float32[189.74646 190.82523 … 157.60829 176.0743; 209.9659 212.12804 … 177.1005 194.7618; … ; 162.9584 161.9159 … 139.23557 155.63895; 172.51736 172.27759 … 142.34506 160.81467]

Float32[171.69823 177.72382 … 153.06985 162.9905; 193.61528 193.97125 … 182.04813 189.34122; … ; 155.76425 166.25867 … 118.48872 138.94125; 157.66699 168.23885 … 127.22163 142.75288]

...

Float32[182.06067 173.12015 … 143.76915 167.41202; 176.6009 172.58246 … 133.63158 157.16907; … ; 195.83804 189.8789 … 163.70184 186.47171; 188.433 178.16035 … 152.87718 176.6781]

Float32[200.65018 191.21686 … 156.33752 183.6864; 200.75879 199.60783 … 150.13602 177.56674; … ; 188.10811 177.26044 … 163.0398 182.86575; 195.28627 181.95357 … 159.72276 184.92444]

Float32[208.06142 200.78818 … 158.69118 187.78445; 216

In [14]:
@view_image result.x

PyObject <napari.viewer.Viewer object at 0x7f5d73e1e6a0>

In [12]:
mip = maximum(res, dims=3)[:, :]

LoadError: UndefVarError: res not defined